## Downscaleing task: Era5 to Daymet 15 to 3.75 arcmin

Analysis code to implement the evaluation of downscaling. Before runnning this code, please make sure
- you have already run inference script `inference_era5_daymet.py` to create inferenced dataset
- make sure the script created files `groundtruth_0000.npy`, ... and `prediction_0000.npy`


- RMSE
    - RMSE for mean, $\sigma_1, \sigma_2, \sigma_3$
- R2
- SSIM
- Fourier diagram distance
- PDF image:0
- Scatter plot image:0
- Taylor diagram

In [ ]:
import os
import copy
import pandas as pd
import glob
import numpy as np
import scipy.stats as stats
from scipy.stats import gaussian_kde
from skimage.metrics import structural_similarity as ssim # you may need to download skimage
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import colors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap

#NOTE you may need to download cartopy besides `pip install -e .`
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
%load_ext autoreload
%autoreload 2
from taylor import TaylorDiagram
from metrics import *

In [ ]:
%load_ext autoreload
%autoreload 2
from psd import psd

In [ ]:
VARIABLE_NAMES= {
    "prcp" : 'prcp'
    # or "tmin": "tmin"
}

Lat and Lon from 3.75arcmin

In [ ]:
def get_lat_weight(latitudes):
    # Convert latitudes to radians and compute weights
    lat_radians = np.deg2rad(latitudes)
    weights = np.cos(lat_radians).clip(0., 1.)
    print("Mean", np.mean(weights))
    return weights

Get latlon data

In [ ]:
lats = np.load("/Users/ttt/Documents/projects/super-resolution/inference_era2daymet/15arcmin-3.75arcmin/lat.npy")

In [ ]:
lat_weights = get_lat_weight(lats)
lat_weights = lat_weights[..., np.newaxis]
lat_weights.shape

In [ ]:
lons = np.load("/Users/ttt/Documents/projects/super-resolution/inference_era2daymet/15arcmin-3.75arcmin/lon.npy")

### Load dataset

In [ ]:
# directory to your inference results
basedatadir = "/Users/ttt/Documents/projects/super-resolution/inference_era2daymet/15arcmin-3.75arcmin"

In [ ]:
variables = ['prcp']
loss_metrics =  ['mse', 'imagegradient'] 
experiment_pairs = {
    "imagegradient": ['3128692'],
    "mse": ['3129425']
}
# assuming that the *.npy data is stored ...../{loss_metrics}/{variable}/{experiment_pairs}/test/*.npy

In [ ]:
import time
def get_data(truth_or_pred):
    Var = {}
    s1 = time.time()
    for var in variables:
        for l in loss_metrics:
            expnames = experiment_pairs[l]
            for expname in expnames:
                print(f"START: {var} {l} {expname} | {(time.time()-s1)/60.} [m]")
                filelist = sorted(glob.glob(
                    os.path.join(*[basedatadir, l, var, expname, 'test',  f"{truth_or_pred}*.npy"])))
                fname = os.path.join(*[basedatadir, l, var, expname, 'test', f"{truth_or_pred}*.npy"])
                  
                #assert len(filelist) > 0, f"File Not Found {fname}"
                if len(filelist) > 0:

                    data_array = None
                    for idx, ifile in enumerate(filelist):
                        data = np.load(ifile).astype(np.float32)   
                    
                        if idx > 0:
                            data_array = np.concatenate([data_array, data], axis=0, dtype=np.float32)
                        else:
                            data_array = data
            
                    Var[f'{var}-{l}-{expname}'] = data_array
    return Var

In [ ]:
# Predict data
Preds = {}
Preds = get_data('prediction')


In [ ]:
Truths = {}
Truths = get_data('groundtruth')

In [ ]:
def quantile_rmse(x, y, q):
    """
        x: pred
        y: truth 
        q: 0 - 1. 1,2,3 sigma = 0.6827, 0.9545, 0.9973  
    """
    #0.6827, 0.9545, 0.9973
    index = np.where(y>=np.quantile(y, q))
    rmse =  np.sqrt(np.mean(np.square(x[index] -  y[index] )))
    return rmse

In [ ]:
def normalize(float_array,vmax, vmin ):
    # Normalize the array to range [0, 1]
    norm_array = (float_array - vmin) / (vmax - vmin)

    # Scale and convert to integers in range [0, 255]
    int_array = (norm_array * 255).astype(np.uint8)
    return int_array

In [ ]:
Metrics = {}
for (k, preds), (_, truths) in zip(Preds.items(),  Truths.items()):
     
    corrs = np.array([])
    wrmses = np.array([])
    s1rmses = np.array([])
    s2rmses = np.array([])
    s3rmses = np.array([])
    ssim_scores = np.array([])
    psnr_scores = np.array([])
    
    for pred, truth in zip(preds, truths ):
        
        corr = clim_pearsoner(x_sim=pred, x_obs=truth )
        wrmse = lat_weight_rmse(x_sim=pred, x_obs=truth, lat_weights=lat_weights)
        corrs = np.append(corrs, corr) 
        wrmses = np.append(wrmses, wrmse)
        
        # >1, 2, 3 sigma rmses
        s1, s2, s3 = 0.6827, 0.9545, 0.9973
        s1rmse = quantile_rmse(pred, truth, s1)
        s2rmse = quantile_rmse(pred, truth, s2)
        s3rmse = quantile_rmse(pred, truth, s3)
        s1rmses = np.append(s1rmses, s1rmse)
        s2rmses = np.append(s2rmses, s2rmse)
        s3rmses = np.append(s3rmses, s3rmse)
        
        # transformation
        vmin = min(np.nanmin(pred), np.nanmin(truth))
        vmax = max(np.nanmax(pred), np.nanmax(truth))
        pred= normalize(pred, vmax=vmax, vmin=vmin)
        truth= normalize(truth, vmax=vmax, vmin=vmin)  
    
        # calc
        _ssim = ssim(pred, truth)
        _psnr = psnr(pred,truth)
        ssim_scores = np.append(ssim_scores, _ssim)
        psnr_scores = np.append(psnr_scores, _psnr)
        
    
    # Add mean
    corr_mean =  np.mean(corrs)
    wrmse_mean= np.mean(wrmses)
    ssim_mean = np.mean(ssim_scores)
    psnr_mean = np.mean(psnr_scores)
    s1rmse_mean = np.mean(s1rmses)
    s2rmse_mean = np.mean(s2rmses)
    s3rmse_mean = np.mean(s3rmses)
    
    # store at dict
    Metrics[k] = {
        'corr': corr_mean,  'rmse': wrmse_mean, 
        "rmse_sigma1": s1rmse_mean,
        "rmse_sigma2": s2rmse_mean,
        "rmse_sigma3": s3rmse_mean,
        'ssim': ssim_mean, 'psnr': psnr_mean
    }

In [ ]:
pd.DataFrame(Metrics).T

##### Calculate PSD

In [ ]:
nsamples = 100 #NOTE: max 365 i.e. the length of 2021

In [ ]:
def calc_RALSD(truths, preds, nsamples=-1):
    return np.sqrt(np.mean(np.square(10*np.log(psd(truths[:nsamples])[0].mean() / 
                                    psd(preds[:nsamples])[0].mean()))) )

In [ ]:
RALSDs = {}
for (k, truths), (_, preds) in zip(Truths.items(), Preds.items()):
    s1 = time.time()
    RALSDs[k] = calc_RALSD(truths, preds, nsamples=nsamples)
    print(f"DONE {k} {(time.time() - s1)/60.0 } [min]")

#### Visualize fine-tune results

In [ ]:
ncols=3
nrows=1
fts=14
cbar_tick_size=12
cbar_xlabel=12
lons[lons>180] -= 360
lonmin, lonmax = np.min(lons), np.max(lons)
latmin, latmax = np.min(lats), np.max(lats)

fig, axes = plt.subplots(nrows,ncols,figsize=(9.5*3/2, 4*nrows),
                         subplot_kw={'projection': ccrs.PlateCarree(central_longitude=-100)})
axes = axes.flatten()

# Define extracted colors (normalized RGB values)
colors = [
    (1.0, 1.0, 1.0), (0.7, 0.8, 0.75), (0.6, 0.75, 0.7),
    (0.5, 0.7, 0.65), (0.3, 0.8, 0.8), (0.5, 0.6, 0.9), (0.75, 0.5, 0.95),
    (0.85, 0.3, 0.85), (0.6, 0.1, 0.4), (0.2, 0.0, 0.2)
]

# Create custom colormap
custom_cmap = LinearSegmentedColormap.from_list("custom_precipitation", colors)

for ax , tname, V, cbar_title_name in zip(
        axes, ['Truth', 'Scratch (w Soil Moist) ', 'Fine-tune  \n (w.o. soil moist & landcover)'],
        [Truths['prcp-mse-3129425'][0],
         Preds['prcp-mse-3129425'][0], 
         Preds['prcp-imagegradient-3128692'][0]], # Please change this lines
        [r'$log_e(mm/day+1)$', r'$log_e(mm/day+1)$',  r'$log_e(mm/day+1)$']
    ):
    ax.set_extent([lonmin, lonmax, latmin, latmax], crs=ccrs.PlateCarree())
    im = ax.imshow(V, cmap = custom_cmap, transform=ccrs.PlateCarree(), 
              extent=[lonmin, lonmax, latmin, latmax], vmin=0,
              )
    ax.set_title(tname, fontsize=fts)
    ax.coastlines()  #Currently can be one of “110m”, “50m”, and “10m”.
    
    # grid format
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                        draw_labels=True,)
    gl.xlines = False
    gl.ylines = False
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10, 'color': 'black'}
    gl.ylabel_style = {'size': 10, 'color': 'black'}
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=.75)
    cbar.ax.set_xlabel(cbar_title_name, fontsize=cbar_xlabel)
    cbar.ax.tick_params(labelsize=cbar_tick_size)

figdir="./figs"
os.makedirs(figdir, exist_ok=True)
plt.savefig("./figs/prcp_truth-predict_img0_2021_fine-tune-scratch-img_15-3.75arcmin.png", dpi=140)
plt.show()